In [1]:
pip install sentence-transformers;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=6ad209d6c6e86b7af1c5a0a5aa62082b9ffd5c5698773a0eeaed1432d28edfb9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
# Allowing colab to access your google drive, as well as reading and writing in it.
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive


In [3]:
cd drive/MyDrive/TIS

/content/drive/MyDrive/TIS


In [4]:
# Using a model for sentence classification. The model cardiffnlp/twitter-roberta-base-sentiment-latest will give the highest scoring sentiment and its label. The maximum input is only 1064 character, so anything longer has been replaced with label 'na' and score 'na' points. 
from transformers import pipeline
sentiment_task = pipeline("sentiment-analysis", model='cardiffnlp/twitter-roberta-base-sentiment-latest', tokenizer='cardiffnlp/twitter-roberta-base-sentiment-latest', max_length = 3825, truncation = True)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [5]:
# Our data is saved in a tsv file
# The tsv file is made by copying the "Comments" column into an excel sheet and export it as a tsv file
import pandas as pd;
data_df = pd.read_csv('NuSeds test.tsv', sep='\t')

In [6]:
# Getting comments list without empty space and identical comments
comments_list = [*set(data_df.Comments.tolist())]

# Removing comments with multiple empty spaces together
import re
comments_range = range(len(comments_list))
for i in comments_range:
  comments_list[i] = re.sub(' +', ' ', comments_list[i])

# Getting unique list
comments_list_unique = [*set(comments_list)]

In [17]:
comment_range = range(len(comments_list_unique))

# Running the sentiment analysis after checking input length. 
comments_list_sentiment = []
for i in comment_range:
  if len(comments_list_unique[i]) > 1064:
    comments_list_sentiment.append(dict({'comment': comments_list_unique[i], 'label': 'na', 'score': 'nan'}))
    continue
  #print(i, '\t', len(comments_list_unique[i]))
  sentiment_temp = sentiment_task(comments_list_unique[i])[0]
  comments_list_sentiment.append(dict({'comment': comments_list_unique[i], 'label': sentiment_temp['label'], 'score': sentiment_temp['score']}))

In [19]:
# Writing our results in txt file due to the size.
with open('nused_senti_result.txt', 'w') as f:
    for i in comment_range:
        f.write("{} | {} | {}\n".format(comments_list_sentiment[i]['comment'], comments_list_sentiment[i]['label'], comments_list_sentiment[i]['score']))